In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3820
inv_config = set_config_inventory_num(inv_num, ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

In [3]:
import pickle


def write_pages(pages, pages_file):
    with open(pages_file, 'wb') as fh:
        pickle.dump(pages, fh)
    
def read_pages(pages_file):
    with open(pages_file, 'rb') as fh:
        return pickle.load(fh)
    



In [4]:
# if rep_es.es_anno.indices.exists(index='pages'):
#     rep_es.es_anno.indices.delete(index='pages')

In [113]:
inv_num = 3820
inv_num = 3841
inv_num = 3810
inv_num = 3853
inv_num = 3833

inv_nums = [
    3810,
    3820,
    3833,
    3841,
    3853
]

inv_nums = range(3864,3865)

for inv_num in inv_nums:
    query = {
        "bool": {
            "must": [
                {"match": {"metadata.inventory_num": inv_num}},
                {"match": {"type": "index_page"}}
            ]
        }
    }

    pages = rep_es.retrieve_pages_by_type("index_page", inv_num)

    pages_file = f"../../data/pages/index_pages-{inv_num}.pcl"

    write_pages(pages, pages_file)
    len(pages)


total hits: {'value': 152, 'relation': 'eq'} 	hits per scroll: 10


In [5]:
inv_num = 3820
#inv_num = 3841
#inv_num = 3810
#inv_num = 3853
#inv_num = 3833


pages_file = f"../../data/pages/index_pages-{inv_num}.pcl"

pages = read_pages(pages_file)
len(pages)

144

In [5]:
import republic.parser.logical.pagexml_4column_index_parser as index_parser

pages.sort(key=lambda x: x.metadata["page_num"])

for page in pages:
    if page.stats["words"] == 0:
        print("Empty page:", page.id)

pages = [page for page in pages if page.stats["words"] > 0]

prev_page_num = pages[0].metadata["page_num"] - 1
prev_page = None
for pi, page in enumerate(pages):
    if page.metadata["page_num"] != prev_page_num + 1:
        raise IndexError(f'discontinuity in sequence of non-empty index pages from page {prev_page_num} to {page.metadata["page_num"]}')
    prev_page_num = page.metadata["page_num"]
    if prev_page and page.metadata["scan_id"] == prev_page.metadata["scan_id"]:
        if abs(page.coords.width - prev_page.coords.width) > 200:
            print(page.id, prev_page.coords.width, page.coords.width, page.coords.width - prev_page.coords.width)
            print("\t", prev_page.id, prev_page.coords.left, prev_page.coords.right, "\t", prev_page.coords.width)
            print("\t", page.id, page.coords.left, page.coords.right, "\t", page.coords.width)
    prev_page = page
    

NameError: name 'pages' is not defined

In [6]:
import republic.parser.logical.pagexml_4column_index_parser as index_parser

remaining_lines = index_parser.set_column_lines()
prev_main_term = None
inv_config["column_gap"]["gap_pixel_freq_ratio"] = 0.75

data = {
    "main_term": [],
    "sub_lemma": [],
    "date_locator": [],
    "date_locator_day": [],
    "date_locator_month": [],
    "page_locator": [],
    "inventory_num": [],
    "scan_id": [],
    "page_id": []
}

inv_nums = [
    3810,
    3820,
    3833,
    3841,
    3853
]

inv_nums = range(3806,3865)
total = 0
errors = 0

for inv_num in inv_nums:
    pages_file = f"../../data/pages/index_pages-{inv_num}.pcl"
    pages = read_pages(pages_file)
    print(f'parsing {len(pages)} index pages for inv_num', inv_num)
    pages.sort(key = lambda x: x.metadata["page_num"])
    for pi, page in enumerate(pages):
        inv_config["column_gap"]["gap_pixel_freq_ratio"] = 0.75
        #print(pi, pages.index(page), page.id, page.stats)
        if page.stats["words"] == 0 or page.coords is None:
            continue
        last_page = True if page == pages[-1] else False
        try:
            line_types = index_parser.classify_and_filter_lines(page, inv_config, last_page=last_page, debug=False)
            index_columns, extra = index_parser.make_index_page_text_regions(page, line_types, inv_config, debug=False)
            lines, words = 0, 0
            column_groups = index_parser.group_index_columns(index_columns)
        except KeyError:
            line_types = index_parser.classify_and_filter_lines(page, inv_config, last_page=last_page, debug=True)
            print(pages.index(page), page.id, page.stats)
            raise
        except (ValueError, IndexError, TypeError):
            print('ERROR IN PAGE', pi, pages.index(page), page.id, page.stats)
            errors += 1
            continue
        for column_group in column_groups:
            column_lines = index_parser.set_column_lines(column_group)
            sub_lemmas, remaining_lines = index_parser.split_sub_lemmas(column_lines, remaining_lines, prev_main_term)
            prev_main_term = sub_lemmas[-1].main_term if len(sub_lemmas) > 0 else ''
            for sub_lemma in sub_lemmas:
                data["main_term"].append(sub_lemma.main_term)
                data["sub_lemma"].append(sub_lemma.sub_lemma)
                data["date_locator"].append(sub_lemma.date_locator)
                data["date_locator_day"].append(sub_lemma.date_locator_day)
                data["date_locator_month"].append(sub_lemma.date_locator_month)
                data["page_locator"].append(sub_lemma.page_locator)
                data["inventory_num"].append(sub_lemma.metadata["inventory_num"])
                data["scan_id"].append(sub_lemma.metadata["scan_id"])
                data["page_id"].append(sub_lemma.metadata["page_id"])
    total += len(pages)
    print(f'\ttotal: {total}\terrors: {errors}')


parsing 100 index pages for inv_num 3806
	total: 100	errors: 0
parsing 143 index pages for inv_num 3807
	total: 243	errors: 0
parsing 112 index pages for inv_num 3808
	total: 355	errors: 0
parsing 114 index pages for inv_num 3809
	total: 469	errors: 0
parsing 120 index pages for inv_num 3810
	total: 589	errors: 0
parsing 118 index pages for inv_num 3811
ERROR IN PAGE 85 85 NL-HaNA_1.01.02_3811_0050-page-98 {'words': 493, 'lines': 227, 'columns': 10, 'extra': 15}
ERROR IN PAGE 103 103 NL-HaNA_1.01.02_3811_0059-page-116 {'words': 480, 'lines': 194, 'columns': 8, 'extra': 30}
	total: 707	errors: 2
parsing 165 index pages for inv_num 3812
ERROR IN PAGE 17 17 NL-HaNA_1.01.02_3812_0017-page-32 {'words': 481, 'lines': 226, 'columns': 9, 'extra': 13}
ERROR IN PAGE 58 58 NL-HaNA_1.01.02_3812_0037-page-73 {'words': 510, 'lines': 192, 'columns': 11, 'extra': 16}
	total: 872	errors: 4
parsing 162 index pages for inv_num 3813
ERROR IN PAGE 39 39 NL-HaNA_1.01.02_3813_0028-page-54 {'words': 424, 'lin

	total: 5920	errors: 48
parsing 0 index pages for inv_num 3845
	total: 5920	errors: 48
parsing 241 index pages for inv_num 3846
ERROR IN PAGE 64 64 NL-HaNA_1.01.02_3846_0040-page-79 {'words': 546, 'lines': 233, 'columns': 10, 'extra': 13}
ERROR IN PAGE 86 86 NL-HaNA_1.01.02_3846_0051-page-101 {'words': 519, 'lines': 198, 'columns': 9, 'extra': 11}
ERROR IN PAGE 108 108 NL-HaNA_1.01.02_3846_0062-page-123 {'words': 486, 'lines': 244, 'columns': 11, 'extra': 21}
ERROR IN PAGE 116 116 NL-HaNA_1.01.02_3846_0066-page-131 {'words': 490, 'lines': 197, 'columns': 8, 'extra': 10}
ERROR IN PAGE 236 236 NL-HaNA_1.01.02_3846_0126-page-251 {'words': 561, 'lines': 223, 'columns': 8, 'extra': 17}
	total: 6161	errors: 53
parsing 242 index pages for inv_num 3847
	total: 6403	errors: 53
parsing 245 index pages for inv_num 3848
ERROR IN PAGE 142 142 NL-HaNA_1.01.02_3848_0079-page-157 {'words': 282, 'lines': 111, 'columns': 9, 'extra': 8}
	total: 6648	errors: 54
parsing 0 index pages for inv_num 3849
	tota

In [7]:
import pandas as pd

df = pd.DataFrame(data=data)
df
df.main_term.value_counts()

Vrankryk.            3061
Jaaken.              2744
Pasporten.           2705
Pruyssen.            2236
Rusland.             1813
                     ... 
Land,                   1
Advocatenvan heb        1
Kersjes.                1
Herves.                 1
Se. Pieterneevens       1
Name: main_term, Length: 34219, dtype: int64

In [14]:
import datetime

today_file = f'../../data/indices/index_entries-3806-3864-{datetime.date.today().isoformat()}.csv.gz'
latest_file = '../../data/indices/index_entries-3806-3864-latest.csv.gz'
df.to_csv(today_file, compression='gzip', sep='\t', index=False)
df.to_csv(latest_file, compression='gzip', sep='\t', index=False)


In [4]:
import pandas as pd

index_files = [
    '../../data/indices/index_entries-3763-3804-latest.csv.gz',
    '../../data/indices/index_entries-3806-3864-latest.csv.gz'
]

headers_early = [
    'lemma', 'main_term', 'text', 'first_line_id', 'first_line_scan_id',
    'first_line_page_id', 'first_line_column_id', 'last_line_id',
    'last_line_scan_id', 'last_line_page_id', 'last_line_column_id'
]

headers_late = [
    'main_term', 'sub_lemma', 'date_locator', 'date_locator_day',
    'date_locator_month', 'page_locator', 'inventory_num', 'scan_id',
    'page_id'
]

#index_early = pd.read_csv(index_files[0], compression='gzip', sep='\t')
#index_late = pd.read_csv(index_files[1], compression='gzip', sep='\t')

index_early

,lemma,main_term,text,first_line_id,first_line_scan_id,first_line_page_id,first_line_column_id,last_line_id,last_line_scan_id,last_line_page_id,last_line_column_id
0,nee Winter quartieren,nee,"nee Winter quartieren, 918.",NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
1,nee Winter quartieren,nee,aerte tifthop van Gran Brief van felicitatie o...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
2,albadie,albadie,"albadie, Collonel, om tot Brigadier aengeselt ...",NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
3,albadie,albadie,"verfoeekende het Commande- ment van Roermonde,...",NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
4,Ackema Commandeur tot Tseroort,Ackema,"Ackema Commandeur tot Tseroort , 90.",NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
...,...,...,...,...,...,...,...,...,...,...,...
140308,Zouner,Zouner,Zouner om gratificatie voor de Kerk te Hom- bu...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...
140309,Zwitzerse Cantons,Zwitzerse,"Zwitzerse Cantons, klagbten van de Capi- teyne...",NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...
140310,Zwitzerse Cantons,Zwitzerse,Missive van bet Ganton Zurich xwee- gens de re...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...
140311,Zwitzerse Cantons,Zwitzerse,Misive van den Raad van Schaf- bausen weegens ...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...,NL-HaNA_1.01.02_3804_0043,NL-HaNA_1.01.02_3804_0043-page-84,NL-HaNA_1.01.02_3804_0043-column-1355-382-938-...


In [119]:
df.head()

,main_term,sub_lemma,date_locator,date_locator_day,date_locator_month,page_locator,inventory_num,scan_id,page_id
0,Aalbregt.,VX Fygeslagen sijn Rey VW versoek om WES uit d...,16 July,16,Jul,419,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
1,Aardenburg. -AarleRixtel. d' Ablaing vanGiesJe...,Sie op Vlaanderen. Sie op 's Hertogenbosch en ...,4 Mey,4,Mey,266,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
2,Aardenburg. -AarleRixtel. d' Ablaing vanGiesJe...,Eerigt en Commissarissen gaande na Vlaanderen ...,22 dito,22,Mey,305,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
3,Aardenburg. -AarleRixtel. d' Ablaing vanGiesJe...,Rapport van Commissarissen: Aan her Jagtgerigt...,10 July,10,Jul,407,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
4,Aardenburg. -AarleRixtel. d' Ablaing vanGiesJe...,Berigt dien aangaande: Commiss,13 Aug.,13,Aug,47.,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007


In [117]:
len(data['main_term'])
for main_term in data['main_term'][:20]:
    print(main_term)

Aalbregt.
Aardenburg. -AarleRixtel. d' Ablaing vanGiesJenburg.
Aardenburg. -AarleRixtel. d' Ablaing vanGiesJenburg.
Aardenburg. -AarleRixtel. d' Ablaing vanGiesJenburg.
Aardenburg. -AarleRixtel. d' Ablaing vanGiesJenburg.
d'AblaingwvanGiessenburg.
d'AblaingwvanGiessenburg.
Ackersdyk.
Acronins.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraal Generaal Admirali-tuiten in 't gemeen. Admiraliteiten in 't gemeen.
Admiraliteit

In [108]:
from republic.helper.metadata_helper import doc_id_to_iiif_url

print(len(pages))
page = pages[54]
inv_config["column_gap"]["gap_pixel_freq_ratio"] = 0.75
print(page.id, page.metadata["page_num"], page.stats)
print(doc_id_to_iiif_url(page.id, margin=0))
print(page.coords.box)
last_page = page == pages[-1]
line_types = index_parser.classify_and_filter_lines(page, inv_config, last_page=last_page, debug=True)
line_count = 0
for line_type in line_types:
    for line in sorted(line_types[line_type], key = lambda x: x.coords.y):
        line_count += 1
        print(f"{line_count}\t{line_type: <16}{line.coords.x: >6}{line.coords.y: >6}\t{len(line.text) if line.text else 0}\t{line.text}")
index_columns, extra = index_parser.make_index_page_text_regions(page, line_types, inv_config, debug=True)
print("NUMBER OF COLUMNS:", len(index_columns))
for column in index_columns:
    print(column.type, column.coords.x, column.coords.box)
    for line in column.lines:
        print(f"\t{line.coords.x: >6}-{line.coords.right}{line.coords.y: >6}{line.coords.bottom: >6}\t{line.text}")

column_groups = index_parser.group_index_columns(index_columns)


194
NL-HaNA_1.01.02_3834_0035-page-69 69 {'words': 500, 'lines': 244, 'columns': 12, 'extra': 23}
https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3834/NL-HaNA_1.01.02_3834_0035.jpg/2400,0,2500,3500/full/0/default.jpg
{'x': 2430, 'y': 309, 'w': 1878, 'h': 3128}
header 7
header	2522-2588	width: 66	top: 391, bottom: 436	E.
header	2845-2914	width: 69	top: 399, bottom: 446	E.
header	3118-3267	width: 149	top: 406, bottom: 447	datums
header	3288-3372	width: 84	top: 409, bottom: 466	Pag
header	3460-3533	width: 73	top: 412, bottom: 461	E.
header	3783-3854	width: 71	top: 416, bottom: 467	E.
header	4042-4189	width: 147	top: 421, bottom: 472	datums
main_term 42
main_term	2974-3159	width: 185	top: 309, bottom: 374	DID
main_term	4212-4308	width: 96	top: 438, bottom: 480	put.
main_term	2474-2597	width: 123	top: 471, bottom: 518	Ee ten,
main_term	3398-3544	width: 146	top: 478, bottom: 546	Eikhof.
main_term	2692-2971	width: 279	top: 640, bottom: 686	door Zanders
main_term	3616-3838	width: 222	top: 662

TypeError: column at index 1 should have type sub_lemma_column but has types: [pagexml_doc, text_region, column, index_column, main_term_column]

In [65]:
a = [1,2,3,4,1,2,3,4]
a.remove(1)
a

[2, 3, 4, 1, 2, 3, 4]

In [177]:
df.shape

(228813, 9)

In [163]:
remaining_lines = index_parser.set_column_lines()
prev_main_term = None
inv_config["column_gap"]["gap_pixel_freq_ratio"] = 0.75

for pi, page in enumerate(pages[0:4]):
    inv_config["column_gap"]["gap_pixel_freq_ratio"] = 0.75
    print(pages.index(page), page.id, page.stats)
    print(doc_id_to_iiif_url(page.id, margin=0))
    if page.stats["words"] == 0 or page.coords is None:
        continue
    last_page = True if page == pages[-1] else False
    line_types = index_parser.classify_and_filter_lines(page, inv_config, last_page=last_page, debug=False)
    #for line_type in line_types:
    #    for line in sorted(line_types[line_type], key = lambda x: x.coords.y):
    #        print(f"\t{line_type: <16}{line.coords.x: >6}{line.coords.y: >6}{line.coords.bottom: >6}\t{line.text}")
    index_columns, extra = index_parser.make_index_page_text_regions(page, line_types, inv_config, debug=False)
    # print("NUMBER OF COLUMNS:", len(index_columns))
    # for column in index_columns:
    #     print(column.type, column.coords.x)
    #    for line in column.lines:
    #        print(f"\t{line.coords.x: >6}{line.coords.y: >6}{line.coords.bottom: >6}\t{line.text}")
    lines, words = 0, 0
    column_groups = index_parser.group_index_columns(index_columns)
    for tr in extra:
        #print(tr.id, tr.stats)
        lines += tr.stats["lines"]
        words += tr.stats["words"]
        #for line in tr.lines:
        #    print(line.text)
    for column_group in column_groups:
        #print("GROUP")
        for column in column_group:
            #print(column.id, column.stats)
            lines += column.stats["lines"]
            words += column.stats["words"]
            #print(column.type)
            #for line in column.lines:
            #    print(f"\t{line.coords.x: >6}{line.coords.y: >6}\t{line.text}")
        column_lines = index_parser.set_column_lines(column_group)
#         for col_type in column_lines:
#             print(col_type, len(column_lines[col_type]))
#             for line in column_lines[col_type]:
#                 print(f'\t{line.coords.left}-{line.coords.right}\t{line.coords.top}\t{line.text}')
        sub_lemmas, remaining_lines = index_parser.split_sub_lemmas(column_lines, remaining_lines, 
                                                                    prev_main_term, debug=False)
        prev_main_term = sub_lemmas[-1].main_term
        #print(len(sub_lemmas), "sub_lemmas")
        for sub_lemma in sub_lemmas:
            if sub_lemma.date_locator_month is None:
                #print("None month:", sub_lemma.date_locator)
                #print(sub_lemma)
                #for col_type in sub_lemma.lines:
                #    for line in sub_lemma.lines[col_type]:
                #        print('\t', col_type, line.text)
                pass
            if sub_lemma.date_locator_month is not None:
                pass
            print(sub_lemma.main_term)
            print('\tdesc:', sub_lemma.sub_lemma)
            print('\tdate:', sub_lemma.date_locator, '\t', sub_lemma.date_locator_day, sub_lemma.date_locator_month)
            print('\tpage:', sub_lemma.page_locator)
            #for col_type in sub_lemma.lines:
            #    #print("sub_lemma col_type", col_type)
            #    for line in sub_lemma.lines[col_type]:
            #        print(col_type, line.text)
#                 print('--------------------\n\n--------------------\n')
        #for col_type in remaining_lines:
        #    for line in remaining_lines[col_type]:
        #        print("\tremaining:", col_type, line.text)
    #print(lines, words)
    #print('\n\n')
    

0 NL-HaNA_1.01.02_3864_0007-page-13 {'words': 322, 'lines': 140, 'columns': 9, 'extra': 18}
https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3864/NL-HaNA_1.01.02_3864_0007.jpg/2400,0,2500,3500/full/0/default.jpg
Aansorgh.
	desc: equeste van Mir. BES. Aansorgh, verzoekende haar Hoog Mogende efficacieuse voorziening tegen de handelingen der Landelyke Vergadering vanCulemborg: Commiss.
	date: 25 Jan. 	 25 Jan
	page: 197
Aansorgh.
	desc: Missive van de Vergadering 's Lands van Culemborg dien aangaande
	date: 2 Febr. 	 2 Feb
	page: 270
Abbema.
	desc: Minister Plenipotentiaris by den Needersaxischen Kreitz. Zie op Hamburg , of Ministers buiten 's Lands.
	date:  	 None None
	page: 
Abescat.
	desc: Requeste van B. Abescat, verzoekende om het Schip de vrouw Margaretha te mogen transpor eeren: Geaccordeerd onder ze kere bepalingen
	date: 28 dito 	 28 dito
	page: 820
Adams.
	desc: Waarnemende de zaken van de vereenigde Staaten van America. Zie op America, of Ministers uitheemsche.
	date:  	 None 

In [158]:
a = [0,1,2,3]
a.pop(-1)
a

[0, 1, 2]

In [138]:
from itertools import combinations

a = [1233, 1492, 2169]

for x1, x2 in combinations(a, 2):
    print(x1, x2)

1233 1492
1233 2169
1492 2169


In [81]:
scan_id = "NL-HaNA_1.01.02_3820_0045"
scan_id = "NL-HaNA_1.01.02_3833_0057"
scan_id = "NL-HaNA_1.01.02_3833_0018"
scan_id = "NL-HaNA_1.01.02_3833_0019"
scan_id = "NL-HaNA_1.01.02_3833_0031"
scan_id = "NL-HaNA_1.01.02_3833_0032"
scan_id = "NL-HaNA_1.01.02_3833_0033"
scan = rep_es.retrieve_scan_by_id(scan_id)
scan.stats

{'lines': 408,
 'words': 1035,
 'text_regions': 45,
 'columns': 0,
 'extra': 0,
 'pages': 0}

In [82]:
from republic.parser.pagexml import republic_pagexml_parser as page_parser

#scan = rep_es.retrieve_scan_by_id("NL-HaNA_1.01.02_3841_0300")
print(scan.id, scan.coords.box)
#scan.metadata["scan_width"] = 4580
for tr in scan.text_regions:
    print(tr.id, tr.stats)
    print("\t", tr.type)
    for line in tr.lines:
        print(f'\t\t{line.coords.left}-{line.coords.right}\t{line.coords.y}\t{line.text}')

NL-HaNA_1.01.02_3833_0033 {'x': 0, 'y': 0, 'w': 4571, 'h': 3751}
NL-HaNA_1.01.02_3833_0033-text_region-1272-3421-9-32 {'lines': 1, 'words': 1, 'text_regions': 0}
	 ['pagexml_doc', 'text_region', 'resolution', 'main']
		1263-1343	3414	vo;
NL-HaNA_1.01.02_3833_0033-text_region-2193-3387-12-36 {'lines': 1, 'words': 1, 'text_regions': 0}
	 ['pagexml_doc', 'text_region', 'resolution', 'main']
		2180-2265	3378	865
NL-HaNA_1.01.02_3833_0033-text_region-1997-3382-34-36 {'lines': 1, 'words': 2, 'text_regions': 0}
	 ['pagexml_doc', 'text_region', 'resolution', 'main']
		1993-2144	3352	17 dito
NL-HaNA_1.01.02_3833_0033-text_region-939-3382-181-71 {'lines': 3, 'words': 4, 'text_regions': 0}
	 ['pagexml_doc', 'text_region', 'resolution', 'main']
		629-1026	3407	Resolutie deswegens-
		1120-1220	3412	dito
		1010-1055	3410	-
NL-HaNA_1.01.02_3833_0033-text_region-1283-3284-24-32 {'lines': 1, 'words': 1, 'text_regions': 0}
	 ['pagexml_doc', 'text_region', 'resolution', 'main']
		1265-1352	3256	324
NL-Ha

In [86]:
from republic.helper.metadata_helper import get_per_page_type_index

inv_metadata = rep_es.retrieve_inventory_metadata(inv_num)
page_type_index = get_per_page_type_index(inv_metadata)

#scan_pages = page_parser.split_scan_pages(scan)
scan_pages = page_parser.split_pagexml_scan(scan, page_type_index, debug=True)
for page in scan_pages:
    print(page.id, page.coords.box)
    print("\t", page.stats)
    for tr in page.extra:
        print('\tEXTRA', tr.id)
        for line in tr.lines:
            print("\t\t", line.coords.x, line.coords.y, line.text)
    for tr in page.text_regions:
        print('\tPAGE-TR', tr.id)
        for line in tr.lines:
            print("\t\t", line.coords.x, line.coords.y, line.text)
    for column in page.columns:
        print("\tCOLUMN", column.id)
        for line in column.lines:
            print("\t\t", line.coords.x, line.coords.y, line.text)
        for tr in column.text_regions:
            print("\t\tCOL-TR", column.id)
            for line in tr.lines:
                print("\t\t\t", line.coords.x, line.coords.y, line.text)


WIDE TEXT REGION: NL-HaNA_1.01.02_3833_0033-text_region-426-363-1763-3094
WIDE TEXT REGION: NL-HaNA_1.01.02_3833_0033-text_region-2185-278-2024-3150
SPLITTING COLUMN BECAUSE IT IS TOO WIDE
lines: 166 gap_pixel_freq_ratio: 0.2
freq_threshold: 12
COLUMN RANGES: [{'start': 653, 'end': 1236}, {'start': 1564, 'end': 2015}]
NUM COLUMNS: 2
EXTRA LINES BEFORE: 24
COLUMN: NL-HaNA_1.01.02_3833_0033-column-633-423-625-2995
	LINE 852-921	423	E.
	LINE 1104-1249	427	datums
	LINE 691-1100	483	Jard Thy Hannes,
	LINE 696-1102	543	versoekende dat den
	LINE 696-1100	583	Heer en Regenten van
	LINE 694-1098	626	Empel en Meerwyk
	LINE 691-1100	670	mogen werden gelast
	LINE 694-1100	706	om een Bedienaar van
	LINE 691-1098	744	de Justitie aan te stellen
	LINE 691-1098	788	en te onderhouden :
	LINE 694-883	825	Commiss-.
	LINE 1095-1249	806	31 July
	LINE 651-1100	888	Requeste van den Heer
	LINE 696-1098	947	van Empel en Meer-
	LINE 696-1098	987	wyk, versoek ende om
	LINE 691-1098	1027	een substitut Secre aris
	

			 3285 320 Bsseque-
			 3303 407 bo.
	COLUMN NL-HaNA_1.01.02_3833_0033-column-3479-284-458-2752
		COL-TR NL-HaNA_1.01.02_3833_0033-column-3479-284-458-2752
			 3675 284 FE.
			 3479 331 Mifiive van den Com-
			 3519 406 mandeur te Demerary,
			 3519 448 kennis geevende dat
			 3519 487 een Kaper van Darba-
			 3519 528 dos met eenig gewa-
			 3519 567 pend Volk een Aine-
			 3519 608 ricaansche Schoonder
			 3524 648 uit de Kivier hadde
			 3519 689 genoomen, en van
			 3517 729 het geene desweegens
			 3521 769 hadde verrigt: Over-
			 3519 812 genoomen,
			 3481 845 Resolutie van Holland
			 3519 894 desweegens-
			 3483 932 stem van Zeeland
			 3483 970 Resolutie deswegens -
			 3483 1011 Antwoord van Bewind-
			 3519 1053 hebberen der Westin-
			 3517 1092 dische Compagnie op.
			 3517 1132 die Resolutie-
			 3481 1180 Resolutie van Holland
			 3519 1253 omtrent de precautien
			 3517 1296 teegens het diverteeren
			 3519 1336 der Producten van ge-
			 3519 1376 hypothequeerde Pl

In [74]:
page.id
for col in page.columns:
    print(col.id)
    for tr in col.text_regions:
        for line in tr.lines:
            print(f'\t{line.coords.left}-{line.coords.right}', line.coords.y, line.text)

NL-HaNA_1.01.02_3833_0032-column-2341-414-191-2594
	2395-2465 414 E.
	2341-2487 468 Eethen.
	2341-2499 741 Egtbert.
	2348-2496 935 Exkbofs
	2348-2501 1905 Eliason.
	2352-2474 2101 Elliot,
	2370-2512 2444 vanden
	2352-2476 2511 Bisen.
	2359-2532 2922 Eyseneur.
NL-HaNA_1.01.02_3833_0032-column-2526-410-468-3008
	2717-2785 410 E.
	2566-2971 481 van Gansoyen, om
	2564-2974 543 intrekking van die Re-
	2562-2974 582 Tolutie en verleende
	2564-2974 623 lurcheantie: Afgesla-
	2564-2656 668 gen
	2526-2971 770 Pasport ad omnes Po-
	2562-2672 825 pulsos
	2528-2976 941 Requeste, versoekendo
	2566-2974 987 aangesteld te werden
	2566-2974 1028 tot haar Hoog Mog,
	2571-2976 1068 Commissaris te Havre
	2571-2976 1108 de Grace: Overge-
	2568-2737 1148 noomen
	2532-2978 1181 Requeste van verscheide
	2573-2978 1227 Kooplieden, versoe-
	2571-2978 1268 kende dat den selve
	2571-2976 1307 daar toe werde aan
	2568-2978 1347 gesteld: Overgenoo-
	2571-2670 1390 men
	2535-2978 1422 Resolutie van Holland
	2573-29

In [207]:
page = pages[14]
print(page.stats)
num_words = 0
count_words = 0
for extra in page.extra:
    print(extra.id, extra.stats)
    num_words += extra.stats["words"]
    for line in extra.lines:
        line_num_words = len(line.text.split(' '))
        print('\t', line_num_words, line.coords.left, line.coords.right, line.text)
        count_words += line_num_words
    print('running count:', count_words)
for col in page.columns:
    print(col.id)
    print(col.stats)
    if len(col.text_regions) == 0:
        num_words += col.stats["words"]
        for line in col.lines:
            line_num_words = len(line.text.split(' '))
            print('\t', line_num_words, line.coords.left, line.coords.right, line.text)
            count_words += line_num_words
    print('running count:', count_words)
    for tr in col.text_regions:
        print(tr.id, tr.stats)
        num_words += tr.stats["words"]
        for line in tr.lines:
            line_num_words = len(line.text.split(' '))
            print('\t', line_num_words, line.coords.left, line.coords.right, line.text)
            count_words += line_num_words
    print('running count:', count_words)

print("num words:", num_words)

{'words': 480, 'lines': 204, 'columns': 1, 'extra': 26}
NL-HaNA_1.01.02_3833_0016-text_region-4067-208-283-118 {'lines': 1, 'words': 1, 'text_regions': 0}
	 1 4171 4311 1778
running count: 1
NL-HaNA_1.01.02_3833_0016-extra-2113-306-1272-3056 {'lines': 105, 'words': 233, 'text_regions': 0}
	 1 2271 2370 pag.
	 1 2533 2599 B.
	 1 2862 2930 B.
	 1 3117 3268 datums
	 1 3291 3378 Pae
	 1 2476 2602 Boen,
	 3 2701 3109 Hof van Engeland,
	 1 2285 2372 635
	 3 2701 3109 tot reclameering van
	 2 2492 2657 cum suis.
	 1 2287 2370 0671
	 4 2699 3112 sijn Schip de Vreede,
	 3 2699 3050 en desselfs Lading
	 2 3119 3263 1 Oct.
	 1 3298 3385 726
	 1 2478 2595 Beers.
	 4 2659 3107 Advocaat van de Oost-
	 2 2699 3105 indische Compagnie.
	 4 2699 3107 Sie op Uostindiën, Oost-
	 2 2699 3054 indische Compagnie.
	 1 2287 2377 726
	 1 2492 2553 de
	 1 2471 2602 Boenus.
	 3 2657 3109 Honderd vyftig guldens
	 1 2699 2884 toegelegt
	 2 3098 3261 10 July
	 1 3289 3383 510
	 1 2469 2592 Boeye.
	 3 2662 3107 Verle

In [78]:
trs = page.get_inner_text_regions()
sum([tr.stats["words"] for tr in trs])
page.stats["words"]

491

In [16]:
response = rep_es.es_anno.get(index="pages", id="NL-HaNA_1.01.02_3820_0008-page-14")
page_json = response["_source"]

In [24]:
import republic.helper.pagexml_helper as pagexml

page = pagexml.json_to_pagexml_page(page_json)
page.stats

IN COLUMN NL-HaNA_1.01.02_3820_0008-text_region-441-351-871-2904 {'lines': 93, 'words': 242, 'text_regions': 0}
IN COLUMN NL-HaNA_1.01.02_3820_0008-column-441-351-871-2904 {'lines': 93, 'words': 242, 'text_regions': 1}
IN COLUMN NL-HaNA_1.01.02_3820_0008-text_region-839-233-998-136 {'lines': 1, 'words': 3, 'text_regions': 0}
IN COLUMN NL-HaNA_1.01.02_3820_0008-column-839-233-998-136 {'lines': 1, 'words': 3, 'text_regions': 1}
IN COLUMN NL-HaNA_1.01.02_3820_0008-text_region-1342-353-855-2938 {'lines': 92, 'words': 253, 'text_regions': 0}
IN COLUMN NL-HaNA_1.01.02_3820_0008-column-1342-353-855-2938 {'lines': 92, 'words': 253, 'text_regions': 1}
IN COLUMN NL-HaNA_1.01.02_3820_0008-text_region-1348-439-109-119 {'lines': 1, 'words': 1, 'text_regions': 0}
IN COLUMN NL-HaNA_1.01.02_3820_0008-column-1348-439-109-119 {'lines': 1, 'words': 1, 'text_regions': 1}
IN PAGE NL-HaNA_1.01.02_3820_0008-column-441-351-871-2904 {'lines': 93, 'words': 242, 'text_regions': 1}
IN PAGE NL-HaNA_1.01.02_3820_00

{'words': 514, 'lines': 199, 'columns': 4, 'extra': 13}

In [30]:
import json

print(json.dumps(rep_es.retrieve_inventory_metadata(3841), indent=4))

{
    "inventory_uuid": "47881e95-07b9-4c17-8cf4-b55a034c8db2",
    "inventory_num": 3841,
    "series_uuid": "6e8998f6-cc66-43b9-92ae-85423ed66958",
    "series_name": "NL-HaNA_1.01.02",
    "inventory_index": "republic_inventory",
    "inventory_doc_type": "inventory",
    "period_start": "1783-07-01",
    "period_end": "1783-12-31",
    "year": 1783,
    "num_pages": 802,
    "title_page_nums": [
        5,
        7,
        571,
        579
    ],
    "sections": [
        {
            "start": 7,
            "end": 569,
            "page_type": "resolution_page",
            "text_page_num": 1
        },
        {
            "start": 571,
            "end": 578,
            "page_type": "respect_page"
        },
        {
            "start": 579,
            "end": 802,
            "page_type": "index_page"
        }
    ],
    "type_page_num_offsets": [
        {
            "page_type": "resolution_page",
            "page_num_offset": 7
        },
        {
            "pag

In [29]:
inv_metadata = {
    "inventory_uuid": "47881e95-07b9-4c17-8cf4-b55a034c8db2",
    "inventory_num": 3841,
    "series_uuid": "6e8998f6-cc66-43b9-92ae-85423ed66958",
    "series_name": "NL-HaNA_1.01.02",
    "inventory_index": "republic_inventory",
    "inventory_doc_type": "inventory",
    "period_start": "1783-07-01",
    "period_end": "1783-12-31",
    "year": 1783,
    "num_pages": 802,
    "title_page_nums": [
        5,
        7,
        571,
        579
    ],
    "sections": [
        {
            "start": 7,
            "end": 569,
            "page_type": "resolution_page",
            "text_page_num": 1
        },
        {
            "start": 571,
            "end": 578,
            "page_type": "respect_page"
        },
        {
            "start": 579,
            "end": 802,
            "page_type": "index_page"
        }
    ],
    "type_page_num_offsets": [
        {
            "page_type": "resolution_page",
            "page_num_offset": 7
        },
        {
            "page_type": "respect_page",
            "page_num_offset": 571
        },
        {
            "page_type": "index_page",
            "page_num_offset": 579
        }
    ],
    "num_scans": 403,
    "created": "2019-07-23T09:11:37.198201",
    "updated": "2021-12-02T09:02:43.201531",
    "page_num_interventions": None
}

rep_es.index_inventory_metadata(inv_metadata)

In [40]:
for page_num in range(565, 580):
    page = rep_es.retrieve_page_by_page_number(page_num, inventory_num=3841)
    print(page.id, page.type[-1], page.stats)
    page_type = None
    for section in inv_metadata["sections"]:
        if page_num in range(section["start"], section["end"]+1):
            page_type = section["page_type"]
    if page_type is None:
        page_type = "empty_page"
    if page.type[-1] != "title_page":
        page.remove_type(page.type[-1])
    page.add_type(page_type)
    print("\tshould be:", page.type)
    rep_es.index_page(page)


total hits: {'value': 1, 'relation': 'eq'} 	hits per scroll: 1
NL-HaNA_1.01.02_3841_0283-page-565 resolution_page {'words': 673, 'lines': 116, 'columns': 2, 'extra': 7}
	should be: ['pagexml_doc', 'text_region', 'page', 'resolution_page']
total hits: {'value': 1, 'relation': 'eq'} 	hits per scroll: 1
NL-HaNA_1.01.02_3841_0284-page-566 empty_page {'words': 744, 'lines': 121, 'columns': 2, 'extra': 5}
	should be: ['pagexml_doc', 'text_region', 'page', 'resolution_page']
total hits: {'value': 1, 'relation': 'eq'} 	hits per scroll: 1
NL-HaNA_1.01.02_3841_0284-page-567 respect_page {'words': 738, 'lines': 125, 'columns': 2, 'extra': 6}
	should be: ['pagexml_doc', 'text_region', 'page', 'resolution_page']
total hits: {'value': 1, 'relation': 'eq'} 	hits per scroll: 1
NL-HaNA_1.01.02_3841_0285-page-568 respect_page {'words': 732, 'lines': 122, 'columns': 2, 'extra': 11}
	should be: ['pagexml_doc', 'text_region', 'page', 'resolution_page']
total hits: {'value': 1, 'relation': 'eq'} 	hits per s

In [34]:
import republic.parser.pagexml.republic_pagexml_parser as pagexml_parser
from republic.download.text_repo import TextRepo
from settings import text_repo_url

text_repo = TextRepo(text_repo_url)

scan_id = "NL-HaNA_1.01.02_3833_0567"

versions = text_repo.get_file_type_versions(scan_id, 'pagexml')

docs = []
for version in versions["items"]:
    #print(version)
    version_metadata = text_repo.get_version_metadata(version['id'])
    print(version_metadata)
    if version_metadata['pim:transcription:transcriber'] != 'CustomTesseractPageXML':
        continue
    print(version['createdAt'], version_metadata['pim:transcription:transcriber'], version['id'])
    scan_pagexml = text_repo.get_content_by_version_id(version['id'])
    filename = f"{scan_id}.xml"
    scan_doc = pagexml_parser.get_scan_pagexml(filename, pagexml_data=scan_pagexml)
    docs.append(scan_doc)

{'pim:transcription:analyzed': '1627941211898', 'pim:transcription:transcriber': 'Transkribus', 'pim:transcription:uuid': 'd26ed4a4-276d-430a-9fb0-127503fe21e5', 'pim:transcription:version': ''}
{'pim:transcription:analyzed': '1616925808000', 'pim:transcription:transcriber': 'CustomTesseractPageXML', 'pim:transcription:uuid': '1956d233-0af5-47db-9edf-2d16bda28cfd', 'pim:transcription:version': ''}
2022-02-16T12:32:31 CustomTesseractPageXML ff9529e5-e65e-460e-8cae-4bdaeb8c12f7
{'pim:transcription:analyzed': '1617005008000', 'pim:transcription:transcriber': 'CustomTesseractPageXML', 'pim:transcription:uuid': '2ef6bb99-3ea7-4325-8813-4bbc5c10477d', 'pim:transcription:version': ''}
2021-04-08T18:41:42 CustomTesseractPageXML 3de5508f-143c-4ac7-b075-a1e5b9ccef97
{'pim:transcription:analyzed': '1612422960000', 'pim:transcription:transcriber': 'CustomTesseractPageXML', 'pim:transcription:uuid': 'c65853f7-9f61-4bb9-8777-e51fbd396814', 'pim:transcription:version': ''}
2021-04-03T20:18:29 CustomT

In [33]:
docs[0].metadata

{'Creator': 'P2PaLA-PRHLT',
 'Created': datetime.datetime(2021, 3, 28, 12, 3, 28),
 'LastChange': datetime.datetime(2021, 3, 28, 12, 3, 28),
 'scan_height': 3772,
 'scan_width': 4613,
 'filename': 'NL-HaNA_1.01.02_3833_0567.xml',
 'series_name': 'NL-HaNA_1.01.02',
 'series_uuid': '6e8998f6-cc66-43b9-92ae-85423ed66958',
 'inventory_uuid': '93b95c12-1805-42f5-98c6-c352681b46bb',
 'inventory_num': 3833,
 'inventory_year': 1778,
 'inventory_period_start': '1778-01-01',
 'inventory_period_end': '1778-12-31',
 'scan_file': 'NL-HaNA_1.01.02_3833_0567.xml',
 'scan_num': 567,
 'ocr_type': 'pagexml',
 'type': 'scan',
 'id': 'NL-HaNA_1.01.02_3833_0567',
 'viewer_url': 'https://images.diginfra.net/framed3.html?imagesetuuid=93b95c12-1805-42f5-98c6-c352681b46bb&uri=https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3833/NL-HaNA_1.01.02_3833_0567.jpg',
 'jpg_url': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3833/NL-HaNA_1.01.02_3833_0567.jpg',
 'iiif_url': 'https://images.diginfra.net/iiif/NL-HaN